<a href="https://colab.research.google.com/github/unimama/learning/blob/master/AppToTranscribeVideoToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# 🎙️ TRANSCRIÇÃO AUTOMÁTICA COM WHISPER
# Execute cada bloco separadamente no Google Colab
# =============================================================================

# BLOCO 1: Verificar ambiente
import sys, platform
print("🐍 Python:", sys.version)
print("💻 Plataforma:", platform.platform())
print("\n✅ Ambiente pronto para começar!")

# =============================================================================
# BLOCO 2: Instalação (Execute uma vez)
print("🔧 Instalando dependências...")
!pip -q install git+https://github.com/openai/whisper.git
!pip -q install torch --index-url https://download.pytorch.org/whl/cu118
!apt -q update && apt -q install -y ffmpeg
print("✅ Instalação concluída!")

# =============================================================================
# BLOCO 3: Upload do arquivo
from google.colab import files
import os

print("📤 Clique no botão abaixo para enviar seu arquivo:")
uploaded = files.upload()

# Pega o primeiro arquivo enviado
input_path = list(uploaded.keys())[0]
file_size_mb = os.path.getsize(input_path) / 1e6

print(f"\n✅ Arquivo recebido: {input_path}")
print(f"📊 Tamanho: {file_size_mb:.2f} MB")
print(f"🎯 Pronto para transcrever!")

# =============================================================================
# BLOCO 4: Transcrição principal
import whisper
import re

print("🤖 Carregando modelo Whisper...")
# Modelos: tiny, base, small, medium, large
MODEL_NAME = "medium"  # Mude para "small" se quiser mais rápido
model = whisper.load_model(MODEL_NAME)
print(f"✅ Modelo '{MODEL_NAME}' carregado!")

print("\n🎙️ Iniciando transcrição...")
result = model.transcribe(input_path, language="pt", verbose=False)
print("✅ Transcrição concluída!")

def formato_tempo(seconds: float) -> str:
    """Converte segundos para formato hh:mm:ss"""
    seconds = max(0, float(seconds))
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

# Processar segmentos e quebrar em frases
frases_com_tempo = []
for segmento in result.get("segments", []):
    tempo_inicio = formato_tempo(segmento["start"])
    texto = segmento["text"].strip()

    # Quebrar por frases usando pontuação
    frases = re.split(r'(?<=[.!?])\s+', texto)

    for frase in frases:
        frase = frase.strip()
        if frase:
            frases_com_tempo.append((tempo_inicio, frase))

print(f"\n📝 Total de frases processadas: {len(frases_com_tempo)}")
print("\n🔍 Primeiras 5 frases:")
for i, (tempo, frase) in enumerate(frases_com_tempo[:5]):
    print(f"[{tempo}] {frase}")

# =============================================================================
# BLOCO 5: Adicionar nomes dos falantes
# CONFIGURE AQUI OS NOMES DOS SEUS FALANTES:
FALANTE_1 = "Entrevistador"  # Mude aqui
FALANTE_2 = "Entrevistado"   # Mude aqui

# Estratégia simples: alternar falantes
linhas_finais = []
for i, (tempo, frase) in enumerate(frases_com_tempo):
    # Alterna entre falantes (começando com FALANTE_1)
    falante = FALANTE_1 if i % 2 == 0 else FALANTE_2
    linha = f"[{tempo}] {falante}: {frase}"
    linhas_finais.append(linha)

print(f"✅ Formatação concluída com {len(linhas_finais)} linhas")
print("\n📋 Exemplo do resultado:")
for linha in linhas_finais[:8]:
    print(linha)

# =============================================================================
# BLOCO 6: Exportar e baixar arquivos
import re
from google.colab import files

# Nome base para os arquivos
nome_base = input_path.split('.')[0]

# === ARQUIVO TXT ===
arquivo_txt = f"{nome_base}_transcricao.txt"
conteudo_txt = "\n".join(linhas_finais)

with open(arquivo_txt, "w", encoding="utf-8") as f:
    f.write(conteudo_txt)

print(f"✅ Arquivo TXT criado: {arquivo_txt}")

# === ARQUIVO SRT ===
def parse_linha(linha):
    """Extrai tempo, falante e texto de uma linha"""
    match = re.match(r"\[(\d{2}:\d{2}:\d{2})\]\s([^:]+):\s(.+)", linha)
    if not match:
        return None
    tempo_str, falante, texto = match.groups()
    h, m, s = map(int, tempo_str.split(":"))
    tempo_segundos = h*3600 + m*60 + s
    return tempo_segundos, falante, texto

def formato_srt(segundos):
    """Converte segundos para formato SRT (hh:mm:ss,mmm)"""
    h = int(segundos // 3600)
    m = int((segundos % 3600) // 60)
    s = int(segundos % 60)
    ms = int((segundos - int(segundos)) * 1000)
    return f"{h:02}:{m:02}:{s:02},{ms:03}"

# Processar linhas para SRT
entradas_srt = []
for linha in linhas_finais:
    dados = parse_linha(linha)
    if dados:
        entradas_srt.append(dados)

# Gerar arquivo SRT
arquivo_srt = f"{nome_base}_transcricao.srt"
linhas_srt = []
DURACAO_FRASE = 3.0  # segundos por frase

for idx, (inicio, falante, texto) in enumerate(entradas_srt, 1):
    fim = inicio + DURACAO_FRASE
    linhas_srt.append(str(idx))
    linhas_srt.append(f"{formato_srt(inicio)} --> {formato_srt(fim)}")
    linhas_srt.append(f"{falante}: {texto}")
    linhas_srt.append("")

with open(arquivo_srt, "w", encoding="utf-8") as f:
    f.write("\n".join(linhas_srt))

print(f"✅ Arquivo SRT criado: {arquivo_srt}")

# === DOWNLOAD DOS ARQUIVOS ===
print("\n📥 Fazendo download dos arquivos...")
files.download(arquivo_txt)
files.download(arquivo_srt)

print("\n🎉 Transcrição concluída com sucesso!")
print(f"📄 Arquivo TXT: {arquivo_txt}")
print(f"🎬 Arquivo SRT: {arquivo_srt}")

# =============================================================================
# 📋 RESULTADO FINAL
print("\n" + "="*50)
print("📋 RESULTADO FINAL DA TRANSCRIÇÃO:")
print("="*50)
for i, linha in enumerate(linhas_finais[:10]):
    print(linha)
if len(linhas_finais) > 10:
    print(f"... e mais {len(linhas_finais)-10} linhas")
print("="*50)

🐍 Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
💻 Plataforma: Linux-6.1.123+-x86_64-with-glibc2.35

✅ Ambiente pronto para começar!
🔧 Instalando dependências...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.

Saving N005 - 50087151 - ARQUIVO ENTRETENIMENTO - MARILIA GABRIELA - 05.02.mp4 to N005 - 50087151 - ARQUIVO ENTRETENIMENTO - MARILIA GABRIELA - 05.02.mp4

✅ Arquivo recebido: N005 - 50087151 - ARQUIVO ENTRETENIMENTO - MARILIA GABRIELA - 05.02.mp4
📊 Tamanho: 51.48 MB
🎯 Pronto para transcrever!
🤖 Carregando modelo Whisper...


100%|█████████████████████████████████████| 1.42G/1.42G [00:17<00:00, 86.5MiB/s]


✅ Modelo 'medium' carregado!

🎙️ Iniciando transcrição...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 63739/63739 [25:22<00:00, 41.87frames/s]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 Transcrição concluída com sucesso!
📄 Arquivo TXT: N005 - 50087151 - ARQUIVO ENTRETENIMENTO - MARILIA GABRIELA - 05_transcricao.txt
🎬 Arquivo SRT: N005 - 50087151 - ARQUIVO ENTRETENIMENTO - MARILIA GABRIELA - 05_transcricao.srt

📋 RESULTADO FINAL DA TRANSCRIÇÃO:
[00:00:00] Entrevistador: Você é daqueles ingleses que são fanáticos pela realeza do seu país ou não?
[00:00:05] Entrevistado: Você curte a família real?
[00:00:07] Entrevistador: Mais ou menos.
[00:00:08] Entrevistado: Por quê?
[00:00:09] Entrevistador: Não, porque não tinha nada a ver com a família real, não.
[00:00:13] Entrevistado: Sim, mas no geral o povo não tem, admira, acha o máximo, acompanha os casamentos, torce,
[00:00:18] Entrevistador: lê as notícias, você...
[00:00:19] Entrevistado: Não, nunca foi assim, não.
[00:00:22] Entrevistador: Mas você acha razoável ter uma família que é a aula atual?
[00:00:26] Entrevistado: Acho uma coisa legal, não.
... e mais 211 linhas
